In [ ]:
!pip install tensorflow_federated==0.12.0

In [ ]:
import os
import pprint
import tensorflow as tf

In [ ]:
try:
  # Use the %tensorflow_version magic if in colab.
  %tensorflow_version 2.x
except Exception:
  pass

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

In [ ]:
%matplotlib inline
import os,random
os.environ["KERAS_BACKEND"] = "theano"
#os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["THEANO_FLAGS"]  = "device=cpu"
import numpy as np
import theano as th
import theano.tensor as T
from keras.utils import np_utils
import keras.models as models
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.recurrent import LSTM
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.regularizers import *
from keras.optimizers import adam
import matplotlib.pypl

In [ ]:
# Import necessary libraries
import pickle as cPickle  # Python3 uses 'pickle' for the 'cPickle' module from Python2
import numpy as np

# Load the dataset from the pickle file
# Make sure the RML2016.10a_dict.pkl file is downloaded and extracted
# from https://www.deepsig.io/datasets. Adjust the file path as needed.
file_path = "RML2016.10a_dict.pkl"

# Since the pickle file was created in Python2, specify encoding='latin1' to handle it in Python3
with open(file_path, 'rb') as f:
    Xd = cPickle.load(f, encoding="latin1") 

# Extract unique SNRs (Signal-to-Noise Ratios) and modulation schemes from dataset keys
snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1, 0])

# Initialize lists to store data (X) and corresponding labels (modulation scheme, SNR)
X = []
lbl = []

# Iterate over each modulation scheme and SNR
for mod in mods:
    for snr in snrs:
        # Append the signal data (features) for the current modulation scheme and SNR
        X.append(Xd[(mod, snr)])
        
        # Append the corresponding label (modulation scheme, SNR) for each signal
        for i in range(Xd[(mod, snr)].shape[0]):
            lbl.append((mod, snr))

# Stack the list of arrays vertically to create a single feature array
X = np.vstack(X)

# The final arrays: 
# X contains the signal data, and lbl contains the labels (mod, SNR) for each signal


Let's break this code down thoroughly.

### Objective:
This part of the code is focused on extracting and organizing **modulation schemes**, **Signal-to-Noise Ratios (SNRs)**, and **signal data** from a dataset that is structured as a dictionary `Xd`. The goal is to:
- Extract unique modulation schemes and SNRs from the dataset keys.
- Organize signal data (`X`) and their corresponding labels (`lbl`) into arrays.

### Step-by-Step Breakdown:

---

### 1. **Extract Unique SNRs and Modulation Schemes**

#### Code:
```python
snrs, mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1, 0])
```

This single line extracts **unique SNR values** and **modulation schemes** from the keys of the dataset dictionary `Xd`. Here's a detailed explanation:

#### **Key Concepts**:
- **`Xd.keys()`**: 
  The dataset `Xd` is a Python dictionary where the keys are tuples in the format `(modulation_scheme, SNR)`. For example:
  ```python
  Xd.keys() -> [('BPSK', -10), ('BPSK', 0), ('QPSK', 10), ...]
  ```
  The key pairs `(modulation_scheme, SNR)` represent unique combinations of modulation schemes (such as BPSK, QPSK, etc.) and SNR values (such as -10, 0, 10, etc.). The values in `Xd` are arrays containing signal data for each modulation scheme and SNR combination.

#### **How the Code Works**:
- **`map(lambda j: ...)`**:
  The `map()` function is applied to extract both **SNR values** and **modulation schemes**:
  - **When `j = 1`**, it extracts the **SNR values** (from the second element of the tuple).
  - **When `j = 0`**, it extracts the **modulation schemes** (from the first element of the tuple).
  
- **`map(lambda x: x[j], Xd.keys())`**:
  This part of the code maps over the keys of `Xd` (which are tuples like `('BPSK', -10)`), and retrieves the **SNR** (when `j = 1`) or the **modulation scheme** (when `j = 0`).
  - Example for **SNR**: Extracting the second element (SNR) from each tuple: `[-10, 0, 10, ...]`.
  - Example for **modulation scheme**: Extracting the first element (modulation scheme) from each tuple: `['BPSK', 'QPSK', ...]`.

- **`set()`**: 
  The extracted values are converted into a **set** to remove duplicates, ensuring that only unique modulation schemes and SNR values are captured.

- **`sorted()`**: 
  The list of unique modulation schemes and SNR values is sorted to ensure they are in ascending order.

- **`map(..., [1, 0])`**: 
  This part indicates that the `map` function is applied twice:
    - First, it extracts and sorts **SNR values** (when `j = 1`).
    - Second, it extracts and sorts **modulation schemes** (when `j = 0`).

### Final Result:
- **`snrs`**: A sorted list of unique SNR values.
- **`mods`**: A sorted list of unique modulation schemes.

### Example:
- If `Xd.keys()` contains:
  ```python
  [('BPSK', -10), ('BPSK', 0), ('QPSK', 10), ('QPSK', -10)]
  ```
  Then the output would be:
  - `mods = ['BPSK', 'QPSK']`
  - `snrs = [-10, 0, 10]`

---

### 2. **Initialize Empty Lists for Data and Labels**

#### Code:
```python
X = []
lbl = []
```

- **`X`**: This empty list will store the **signal data** (features) for all modulation schemes and SNR values.
- **`lbl`**: This empty list will store the **corresponding labels** for each signal in `X`. Each label will be a tuple `(modulation_scheme, SNR)`.

---

### 3. **Iterate Over Each Modulation Scheme and SNR**

#### Code:
```python
for mod in mods:
    for snr in snrs:
```

- This **nested loop** iterates over all combinations of **modulation schemes** (`mods`) and **SNR values** (`snrs`).

    - **Outer loop**: `mod` takes each modulation scheme from `mods` (e.g., 'BPSK', 'QPSK', etc.).
    - **Inner loop**: `snr` takes each SNR value from `snrs` (e.g., -10, 0, 10, etc.).

For each combination `(mod, snr)`, the corresponding **signal data** is extracted, and labels are created.

---

### 4. **Append Signal Data and Labels**

#### Code:
```python
# Append the signal data (features) for the current modulation scheme and SNR
X.append(Xd[(mod, snr)])

# Append the corresponding label (modulation scheme, SNR) for each signal
for i in range(Xd[(mod, snr)].shape[0]):
    lbl.append((mod, snr))
```

#### **Appending Signal Data**:
- **`Xd[(mod, snr)]`**:
  This retrieves the signal data from the dataset `Xd` for the current modulation scheme (`mod`) and SNR (`snr`). The signal data is stored as a **numpy array**, where:
  - The first dimension of the array represents different signals (examples) for the `(mod, snr)` pair.
  - The next dimensions represent the signal's features (such as I/Q components with time samples).

  For example:
  ```python
  Xd[('BPSK', 0)] -> numpy array with shape (N, 2, 128)
  ```
  Here, `N` is the number of signals (examples) with modulation scheme `BPSK` and SNR `0`.

- **`X.append(Xd[(mod, snr)])`**:
  This appends the entire numpy array for the current `(mod, snr)` pair to the list `X`. Each appended element is a 3D array of signal data with shape `(N, 2, 128)`.

#### **Appending Labels**:
- **`Xd[(mod, snr)].shape[0]`**:
  This gives the number of signals (examples) in the numpy array for the current `(mod, snr)` pair. Let's call this `N`.

- **Labeling Process**:
  For each signal (row) in the array `Xd[(mod, snr)]`, the tuple `(mod, snr)` is appended to the list `lbl`:
  ```python
  for i in range(N):
      lbl.append((mod, snr))
  ```
  This loop runs `N` times, where `N` is the number of signals in `Xd[(mod, snr)]`. Each signal is assigned the label `(mod, snr)`.

### Example:
Suppose there are 100 signals in `Xd[('BPSK', 0)]`, the loop will append the label `('BPSK', 0)` 100 times to `lbl`, corresponding to each of those 100 signals.

---

### 5. **Stack the List of Arrays Vertically**

#### Code:
```python
X = np.vstack(X)
```

At this point:
- **`X`** is a list of 3D arrays, where each array represents the signal data for a specific modulation scheme and SNR (e.g., `Xd[('BPSK', 0)]`).

- **`np.vstack(X)`**:
  - This function stacks all the 3D arrays from the list `X` **vertically**, creating a single large 3D numpy array.
  - After stacking, the final `X` becomes a single numpy array where:
    - The first dimension is the total number of signals (rows).
    - The second and third dimensions are the features of each signal (e.g., `(2, 128)` for I/Q samples).

### Example:
Let’s assume:
- `X` contains 3 arrays:
  - An array of shape `(100, 2, 128)` for `('BPSK', 0)`.
  - An array of shape `(150, 2, 128)` for `('QPSK', 0)`.
  - An array of shape `(200, 2, 128)` for `('8PSK', 10)`.

After stacking, `X` will have the shape:
```python
X.shape -> (450, 2, 128)
```
Where 450 is the total number of signals (100 + 150 + 200).

---

### Final Arrays:

- **`X`**:
  - A 3D numpy array where each signal is represented by its features (e.g., I/Q components with 128 time samples).
  - Shape: `(total_signals, 2, 128)`, where `total_signals` is the total number of signals across all modulation schemes and SNRs.

- **`lbl`**:

  - A list of tuples `(mod, snr)`, where each tuple represents the modulation scheme and SNR for the corresponding signal in `X`.



In [ ]:
X.shape[0]

220000

In [ ]:
# Partition the dataset into training and test sets
# Ensuring that SNR and modulation labels are maintained for each sample

# Set a random seed for reproducibility
np.random.seed(1000)

# Determine the total number of examples
n_examples = X.shape[0]

# Take 75% of the samples for training
n_train = int(n_examples * 0.75)

# Randomly select indices for the training set (without replacement)
train_idx = np.random.choice(range(0, n_examples), size=n_train, replace=False)

# The test set indices are the remaining samples
test_idx = list(set(range(0, n_examples)) - set(train_idx))

# Split the feature data into training and test sets
X_train = X[train_idx]
X_test = X[test_idx]

# Function to convert labels to one-hot encoding
# 'yy' is a list of labels that gets transformed into a one-hot encoded matrix
def to_onehot(yy):
    data = list(yy)
    yy1 = np.zeros([len(data), max(data) + 1])
    yy1[np.arange(len(data)), data] = 1
    return yy1

# Convert training labels to one-hot encoding
# The labels are based on the modulation scheme index corresponding to the training indices
Y_train = to_onehot(map(lambda x: mods.index(lbl[x][0]), train_idx))

# Convert test labels to one-hot encoding using the same logic
Y_test = to_onehot(map(lambda x: mods.index(lbl[x][0]), test_idx))


Let's break down the line `Y_train = to_onehot(map(lambda x: mods.index(lbl[x][0]), train_idx))` in detail:

### 1. **Purpose**:
The goal of this line is to create the one-hot encoded labels for the training data (`Y_train`). Each label represents the modulation scheme for the corresponding signal in `X_train`.

### 2. **Understanding the Components**:

#### `map(lambda x: mods.index(lbl[x][0]), train_idx)`:
- **`train_idx`**: 
  This is the list of indices that have been randomly selected for the training set from the entire dataset. Each index points to a specific sample in the dataset.
  
- **`lbl`**: 
  This list holds the original labels in the form `(modulation_scheme, SNR)` for each signal in the dataset. So, `lbl[x]` is the label for the sample at index `x`, and `lbl[x][0]` extracts the modulation scheme for that sample.

- **`mods.index(lbl[x][0])`**: 
  - `lbl[x][0]` extracts the modulation scheme from the label at index `x`. 
  - `mods` is a list of all modulation schemes (e.g., `['BPSK', 'QPSK', '8PSK', ...]`), and `mods.index(lbl[x][0])` finds the **index** of the modulation scheme in the `mods` list. This transforms the modulation scheme (a string) into a numerical label.

- **`lambda x: mods.index(lbl[x][0])`**: 
  This is an anonymous function (using the `lambda` keyword) that takes an index `x`, finds the corresponding modulation scheme from `lbl[x][0]`, and returns the **index** of that modulation scheme from the `mods` list.

- **`map(lambda x: mods.index(lbl[x][0]), train_idx)`**: 
  The `map` function applies the `lambda` function to each index in `train_idx`. For each index `x`, it retrieves the modulation scheme for the corresponding sample in the dataset, converts it to its numerical index, and returns these indices as a map object (which behaves like a list in Python 3).

### 3. **`to_onehot()` Function**:
- The `to_onehot()` function takes a list of numerical labels (the output of the `map()` function) and converts them into a **one-hot encoded** matrix. 
  - One-hot encoding is a way of representing categorical labels where each category (in this case, modulation scheme) is represented as a binary vector.
  - For example, if there are 5 modulation schemes, a label `2` will be represented as `[0, 0, 1, 0, 0]`.

#### Inside `to_onehot()`:
- It creates a 2D array (`yy1`) with the number of rows equal to the number of labels (one row per sample) and the number of columns equal to the number of classes (one column for each modulation scheme).
- It uses `np.arange()` to get a list of row indices and assigns a value of `1` to the corresponding column index (the modulation scheme) in each row.

### Putting it all together:
- **Step 1**: The `map(lambda x: mods.index(lbl[x][0]), train_idx)` transforms the list of training indices (`train_idx`) into a list of **numerical indices** representing the modulation scheme for each training sample.
- **Step 2**: The `to_onehot()` function takes that list of modulation indices and converts it into a **one-hot encoded matrix**, where each row corresponds to a training sample, and the columns represent the modulation schemes.

### Example:
- Suppose `train_idx = [10, 20, 30]`, and the corresponding modulation schemes for these samples are `QPSK`, `BPSK`, and `8PSK`.
- The `map()` function transforms these schemes into their numerical indices based on their position in `mods`, e.g., `[1, 0, 2]`.
- The `to_onehot()` function will convert these indices into a one-hot encoded array:
  ```
  [[0, 1, 0],  # QPSK (index 1)
   [1, 0, 0],  # BPSK (index 0)
   [0, 0, 1]]  # 8PSK (index 2)
  ```

### Final Output:
`Y_train` will be a matrix where each row is a one-hot encoded vector representing the modulation scheme for the corresponding training sample in `X_train`.

In [ ]:
in_shp = list(X_train.shape[1:])

In [ ]:
print (X_test.shape, in_shp)

(55000, 2, 128) [2, 128]


In [ ]:
print (X_train.shape, in_shp)

(165000, 2, 128) [2, 128]


In [ ]:
Y_train[0]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [ ]:
Y_train=Y_train.astype('int32')

In [ ]:
Y_test=Y_test.astype('int32')

In [ ]:
total_train=len(X_train)

In [ ]:
classes = mods

In [ ]:
classes

['8PSK',
 'AM-DSB',
 'AM-SSB',
 'BPSK',
 'CPFSK',
 'GFSK',
 'PAM4',
 'QAM16',
 'QAM64',
 'QPSK',
 'WBFM']

In [ ]:
CLIENTS=3
BATCH_SIZE_train = int(total_train/3) 
BATCH_SIZE = 1000

In [ ]:
from sklearn.utils import shuffle

In [ ]:
x_train, y_train = shuffle(X_train, Y_train, random_state=1000)
x_test, y_test = shuffle(X_test, Y_test, random_state=1000)

In [ ]:
def generate_clients_datasets(n, BATCH_SIZE_EXT, source_x, source_y):
    clients_dataset=[]
    for i in range(n):
        dataset=tf.data.Dataset.from_tensor_slices((source_x[i*BATCH_SIZE_EXT:(i+1)*BATCH_SIZE_EXT], source_y[i*BATCH_SIZE_EXT:(i+1)*BATCH_SIZE_EXT]))
        dataset = dataset.batch(BATCH_SIZE)
        clients_dataset.append(dataset)
    return clients_dataset

In [ ]:
train_dataset_individual=generate_clients_datasets(CLIENTS, BATCH_SIZE_train, x_train, y_train)

In [ ]:
test_dataset_central=generate_clients_datasets(1, len(x_test), x_test, y_test)

In [ ]:
Epochs=200

In [ ]:
federated_data=train_dataset_individual

In [ ]:
batch_of_samples = tf.nest.map_structure(
    lambda x: x.numpy(), iter(train_dataset_individual[0]).next()
)

In [ ]:
dr = 0.2 # dropout rate (%)

In [ ]:
def model_instance_1():
    """Instantiates the keras model."""
    federated_model = tf.keras.models.Sequential([
                                                  
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(128,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(32, (2, 3), strides=1, padding="valid", input_shape=(1, 2, 128), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
      
    ])
    federated_model.compile(optimizer='SGD',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return tff.learning.from_compiled_keras_model(federated_model, batch_of_samples)

federated_learning_iterative_process_1 = tff.learning.build_federated_averaging_process(
    model_instance_1
)

state_1 = federated_learning_iterative_process_1.initialize()

evaluation_1 = tff.learning.build_federated_evaluation(model_instance_1)

federated_model_1 = tf.keras.models.Sequential([
                                                  
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(128,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(32, (2, 3), strides=1, padding="valid", input_shape=(1, 2, 128), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
      
    ])

In [ ]:
def model_instance_2():
    """Instantiates the keras model."""
    federated_model = tf.keras.models.Sequential([
                                                  
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(192,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64, (2, 3), strides=1, padding="valid", input_shape=(1, 2, 128), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
      
    ])
    federated_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return tff.learning.from_compiled_keras_model(federated_model, batch_of_samples)

federated_learning_iterative_process_2 = tff.learning.build_federated_averaging_process(
    model_instance_2
)

state_2 = federated_learning_iterative_process_2.initialize()

evaluation_2 = tff.learning.build_federated_evaluation(model_instance_2)

federated_model_2 = tf.keras.models.Sequential([
    
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(192,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64, (2, 3), strides=1, padding="valid", input_shape=(1, 2, 128), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
      
      
    ])

In [ ]:
def model_instance_3():
    """Instantiates the keras model."""
    federated_model = tf.keras.models.Sequential([
                                                  
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(256,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(80, (2, 3), strides=1, padding="valid", input_shape=(1, 2, 128), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
      
    ])
    federated_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return tff.learning.from_compiled_keras_model(federated_model, batch_of_samples)

federated_learning_iterative_process_3 = tff.learning.build_federated_averaging_process(
    model_instance_3
)

state_3 = federated_learning_iterative_process_3.initialize()

evaluation_3 = tff.learning.build_federated_evaluation(model_instance_3)

federated_model_3 = tf.keras.models.Sequential([
                                                  
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(256,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(80, (2, 3), strides=1, padding="valid", input_shape=(1, 2, 128), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
    ])

In [ ]:
def model_instance_4():
    """Instantiates the keras model."""
    federated_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64,( 2, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64, (1, 3), strides=1, padding="valid", activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
      
    ])
    federated_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return tff.learning.from_compiled_keras_model(federated_model, batch_of_samples)

federated_learning_iterative_process_4 = tff.learning.build_federated_averaging_process(
    model_instance_4
)

state_4 = federated_learning_iterative_process_4.initialize()

evaluation_4 = tff.learning.build_federated_evaluation(model_instance_4)

federated_model_4 = tf.keras.models.Sequential([
                                              
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64,( 2, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64, (1, 3), strides=1, padding="valid", activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
    

    ])

In [ ]:
def model_instance_5():
    """Instantiates the keras model."""
    federated_model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(128,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(128,( 2, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64, (1, 3), strides=1, padding="valid", activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')
    ])
    federated_model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return tff.learning.from_compiled_keras_model(federated_model, batch_of_samples)

federated_learning_iterative_process_5 = tff.learning.build_federated_averaging_process(
    model_instance_5
)

state_5 = federated_learning_iterative_process_5.initialize()

evaluation_5 = tff.learning.build_federated_evaluation(model_instance_5)

federated_model_5 = tf.keras.models.Sequential([

    tf.keras.layers.Input(shape=(2,128)), 
    tf.keras.layers.Reshape(( 2, 128,1)),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(128,( 1, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(128,( 2, 3), activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.ZeroPadding2D((0, 2)),
    tf.keras.layers.Conv2D(64, (1, 3), strides=1, padding="valid", activation="relu", kernel_initializer='glorot_uniform'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(dr),
    tf.keras.layers.Dense(11, kernel_initializer='he_normal', activation='softmax')


    ])

In [ ]:
from collections import Counter

from sklearn.metrics import classification_report

import time

In [ ]:
fed_sparse_categorical_accuracy_1=[]
fed_sparse_categorical_accuracy_2=[]
fed_sparse_categorical_accuracy_3=[]
fed_sparse_categorical_accuracy_4=[]
fed_sparse_categorical_accuracy_5=[]
fed_loss_1=[]
fed_loss_2=[]
fed_loss_3=[]
fed_loss_4=[]
fed_loss_5=[]
fed_val_sparse_categorical_accuracy_1=[]
fed_val_sparse_categorical_accuracy_2=[]
fed_val_sparse_categorical_accuracy_3=[]
fed_val_sparse_categorical_accuracy_4=[]
fed_val_sparse_categorical_accuracy_5=[]
fed_val_loss_1=[]
fed_val_loss_2=[]
fed_val_loss_3=[]
fed_val_loss_4=[]
fed_val_loss_5=[]
ensemble_test_accuracy=[]

In [ ]:
EPOCHS=330

In [ ]:
for n in range(EPOCHS):
    start_time = time.time()
    
    start_time_2 = time.time()
    state_1, metrics_1 = federated_learning_iterative_process_1.next(state_1, federated_data)
    test_metrics_1 = evaluation_1(state_1.model, [test_dataset_central[0],test_dataset_central[0]])
    print('model {}, round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(1,n+1, metrics_1[1], metrics_1[0], test_metrics_1[1], test_metrics_1[0] ))
    print("--- %s seconds ---" % (time.time() - start_time_2))
    
    start_time_2 = time.time()
    state_2, metrics_2 = federated_learning_iterative_process_2.next(state_2, federated_data)
    test_metrics_2 = evaluation_2(state_2.model, [test_dataset_central[0],test_dataset_central[0]])
    print('model {}, round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(2,n+1, metrics_2[1], metrics_2[0], test_metrics_2[1], test_metrics_2[0] ))
    print("--- %s seconds ---" % (time.time() - start_time_2))
    
    start_time_2 = time.time()
    state_3, metrics_3 = federated_learning_iterative_process_3.next(state_3, federated_data)
    test_metrics_3 = evaluation_3(state_3.model, [test_dataset_central[0],test_dataset_central[0]])
    print('model {}, round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(3,n+1, metrics_3[1], metrics_3[0], test_metrics_3[1], test_metrics_3[0] ))
    print("--- %s seconds ---" % (time.time() - start_time_2))
    
    start_time_2 = time.time()    
    state_4, metrics_4 = federated_learning_iterative_process_4.next(state_4, federated_data)
    test_metrics_4 = evaluation_4(state_4.model, [test_dataset_central[0],test_dataset_central[0]])
    print('model {}, round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(4,n+1, metrics_4[1], metrics_4[0], test_metrics_4[1], test_metrics_4[0] ))
    print("--- %s seconds ---" % (time.time() - start_time_2))
    
    start_time_2 = time.time()    
    state_5, metrics_5 = federated_learning_iterative_process_5.next(state_5, federated_data)
    test_metrics_5 = evaluation_5(state_5.model, [test_dataset_central[0],test_dataset_central[0]])
    print('model {}, round  {}, fed_loss: {} - fed_sparse_categorical_accuracy: {} - fed_val_loss: {}- fed_val_sparse_categorical_accuracy: {}'.format(5,n+1, metrics_5[1], metrics_5[0], test_metrics_5[1], test_metrics_5[0] ))
    print("--- %s seconds ---" % (time.time() - start_time_2))
    
    start_time_2 = time.time()
    fed_sparse_categorical_accuracy_1.append(metrics_1[0])
    fed_sparse_categorical_accuracy_2.append(metrics_2[0])
    fed_sparse_categorical_accuracy_3.append(metrics_3[0])
    fed_sparse_categorical_accuracy_4.append(metrics_4[0])
    fed_sparse_categorical_accuracy_5.append(metrics_5[0])
    
    fed_loss_1.append(metrics_1[1])
    fed_loss_2.append(metrics_2[1])
    fed_loss_3.append(metrics_3[1])
    fed_loss_4.append(metrics_4[1])
    fed_loss_5.append(metrics_5[1])

    
    fed_val_sparse_categorical_accuracy_1.append(test_metrics_1[0])
    fed_val_sparse_categorical_accuracy_2.append(test_metrics_2[0])
    fed_val_sparse_categorical_accuracy_3.append(test_metrics_3[0])
    fed_val_sparse_categorical_accuracy_4.append(test_metrics_4[0])
    fed_val_sparse_categorical_accuracy_5.append(test_metrics_5[0])
    
    fed_val_loss_1.append(test_metrics_1[1]) 
    fed_val_loss_2.append(test_metrics_2[1]) 
    fed_val_loss_3.append(test_metrics_3[1]) 
    fed_val_loss_4.append(test_metrics_4[1]) 
    fed_val_loss_5.append(test_metrics_5[1]) 
    
    tffweights_1 = tff.learning.model_utils.ModelWeights.from_tff_result(state_1.model)
    tffweights_2 = tff.learning.model_utils.ModelWeights.from_tff_result(state_2.model)
    tffweights_3 = tff.learning.model_utils.ModelWeights.from_tff_result(state_3.model)
    tffweights_4 = tff.learning.model_utils.ModelWeights.from_tff_result(state_4.model)
    tffweights_5 = tff.learning.model_utils.ModelWeights.from_tff_result(state_5.model)
    
    tffweights_1.assign_weights_to(federated_model_1)
    tffweights_2.assign_weights_to(federated_model_2)
    tffweights_3.assign_weights_to(federated_model_3)
    tffweights_4.assign_weights_to(federated_model_4)
    tffweights_5.assign_weights_to(federated_model_5)

    y_pred_1_test = federated_model_1.predict(x_test)
    y_pred_3_test = federated_model_3.predict(x_test)
    y_pred_2_test = federated_model_2.predict(x_test)
    y_pred_4_test = federated_model_4.predict(x_test)
    y_pred_5_test = federated_model_5.predict(x_test)
    
    y_pred_bool_1_test = np.argmax(y_pred_1_test, axis=1)
    y_pred_bool_2_test = np.argmax(y_pred_2_test, axis=1)
    y_pred_bool_3_test = np.argmax(y_pred_3_test, axis=1)
    y_pred_bool_4_test = np.argmax(y_pred_4_test, axis=1)
    y_pred_bool_5_test = np.argmax(y_pred_5_test, axis=1)
    
    y_pred_test_total=[]
    
    for i in np.arange(0, len(y_pred_bool_1_test), 1):
      listnew=[]
      listnew.append(y_pred_bool_1_test[i])
      listnew.append(y_pred_bool_2_test[i])
      listnew.append(y_pred_bool_3_test[i])
      listnew.append(y_pred_bool_4_test[i])
      listnew.append(y_pred_bool_5_test[i])
      y_pred_test_total.append(Counter(listnew).most_common()[0][0])
  
    y_pred_test_total=np.array(y_pred_test_total)
    y_test_bool = np.argmax(y_test, axis=1)
    
    ensemble_test_accuracy.append(classification_report(y_test_bool, y_pred_test_total, output_dict=True)['accuracy'])

    print('model -ensemble, round  {}, - fed_val_sparse_categorical_accuracy: {}'.format(n+1, ensemble_test_accuracy[n] ))
    print("--- %s seconds ---" % (time.time() - start_time_2))
    print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
fig, ax = plt.subplots() # create a new figure with a default 111 subplot
ax.plot(t, fed_val_sparse_categorical_accuracy_1,'b-',t,fed_val_sparse_categorical_accuracy_2,'g-',t,fed_val_sparse_categorical_accuracy_3,'r-',t,fed_val_sparse_categorical_accuracy_4,'c-',t, fed_val_sparse_categorical_accuracy_5,'-m',t,ensemble_test_accuracy,'k')
plt.legend([r'$M_1$',r'$M_2$',r'$M_3$',r'$M_4$',r'$M_5$',r'$M_E$'])
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
axins = zoomed_inset_axes(ax, 4, loc=8, bbox_to_anchor=(230,0,1,1), borderpad=6) # zoom-factor: 2.5, location: upper-left
axins.plot(t, fed_val_sparse_categorical_accuracy_1,'b-',t,fed_val_sparse_categorical_accuracy_2,'g-',t,fed_val_sparse_categorical_accuracy_3,'r-',t,fed_val_sparse_categorical_accuracy_4,'c-',t, fed_val_sparse_categorical_accuracy_5,'-m',t,ensemble_test_accuracy,'k')
x1, x2, y1, y2 = 280, 330, 0.80, 0.855 # specify the limits

mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
axins.set_xlim(x1, x2) # apply the x-limits
axins.set_ylim(y1, y2) # apply the y-limits
plt.show()

<p style="text-align:center">
    <img src="results.png" width="600">
</p>